# Домашнее задание к лекции «Основы парсинга и работы с API»

#### Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>


##### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [5]:
import requests
import time
from bs4 import BeautifulSoup

In [6]:
import pandas as pd

In [71]:
KEYWORDS = ['python', 'парсинг']

In [72]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [73]:
posts = soup.find_all('article', class_='post')

In [74]:
links = []

for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
            if any([hub_lower in desired for desired in KEYWORDS]):
                link = post.find('a', class_='post__title_link').attrs.get('href')
                links.append(link)
print(links)
                

['https://habr.com/ru/company/skillfactory/blog/530594/', 'https://habr.com/ru/post/530574/', 'https://habr.com/ru/post/530570/']


In [84]:
hub_art = pd.DataFrame()

for link in links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    date = soup.find('span', class_='post__time').attrs['data-time_published']
    title = soup.find('span', class_='post__title-text').text
    text = soup.find('div', class_='post__text').text
    row = {'date': date, 'title': title, 'link': link, 'text': text}
    hub_art = pd.concat([hub_art, pd.DataFrame([row])])  
hub_art

,date,title,link,text
0,2020-11-30T12:58Z,Как разработать ансамбль Light Gradient Booste...,https://habr.com/ru/company/skillfactory/blog/...,В преддверии старта нового потока курса «Машин...
0,2020-11-30T10:58Z,Deep Anomaly Detection,https://habr.com/ru/post/530574/,Детекция аномалий с помощью методов глубокого ...
0,2020-11-30T10:39Z,Аудит чат-ботов,https://habr.com/ru/post/530570/,\n\r\nВ настоящее время чат-боты широко распро...


#### Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.



In [261]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Content-type': 'application/json;charset=UTF-8', 
               'Host': 'identityprotection.avast.com',
                'Connection': 'keep-alive',
                'Content-Length': '32',
                'Accept': 'application/json, text/plain, */*',
                'Vaar-Version': '0',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
                'Vaar-Header-App-Product': 'hackcheck-web-avast',
                'Origin': 'https://www.avast.com',
                'Sec-Fetch-Site': 'same-site',
                'Sec-Fetch-Mode': 'cors',
                'Sec-Fetch-Dest': 'empty',
                'Referer': 'https://www.avast.com/',
                'Accept-Encoding': 'gzip, deflate, br',
                'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'  }

In [319]:
leakege = pd.DataFrame()

for email in EMAIL:
    params = {"emailAddresses":[email]}
    res = requests.post(URL,data=json.dumps(params),headers=headers)
    time.sleep(0.3)
    answer = json.loads(res.text)
    for item in answer['breaches'].values():
        site = item['site']
        date = item['publishDate']
        description = item['description']
        row = {'email': email, 'date': date, 'site': site, 'description': description}
        leakege = pd.concat([leakege, pd.DataFrame([row])])  
leakege

,email,date,site,description
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,yyy@y.com,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,yyy@y.com,2019-02-21T00:00:00Z,www.dangdang.com,"This is a list of email addresses only, and as..."


##### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [2]:
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get'
TOKEN = 'тут был токен'
GROUP = 'netology'
VERSION = '5.52'
SLEEP = 0.33

In [3]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': GROUP ,
    'count': 50
}

In [7]:
res = requests.get(NEWSFEED_REQUEST, params)
res

<Response [200]>

In [23]:
netology_posts = pd.DataFrame()

for post in res.json()['response']['items']:
    date = post['date']
    text = post['text']
    row = {'date': date, 'text': text}
    netology_posts = pd.concat([netology_posts, pd.DataFrame([row])])  
netology_posts

,date,text
0,1606723210,"В воздухе уже витает дух Нового года, а мы уст..."
0,1606981560,Карьера сегодня — не прямой и понятный путь из...
0,1606916102,У нас сегодня Нето-викторина 🤩\n\nМы подготови...
0,1606893780,🔥 19 декабря пройдёт бесплатная онлайн-конфере...
0,1606834260,"С тире или без? А может, дефис поставить?\n\nР..."
0,1606807380,"Что надо знать и уметь, чтобы работать с данны..."
0,1606744380,📆 Составляем адвент-календарь взрослого челове...
0,1606720020,🔥 Запустили бесплатный курс «Как перенести сво...
0,1606642020,"Неопределённость, непредсказуемые ситуации, ре..."
0,1606573680,Онлайн-мероприятия декабря: конференции для ди...
